In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


#for save pickle
import sys
import os
import pickle
def save_as_pickled_object(obj, filepath):
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj, protocol=pickle.HIGHEST_PROTOCOL)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])


def try_to_load_as_pickled_object_or_None(filepath):
    max_bytes = 2**31 - 1
    try:
        input_size = os.path.getsize(filepath)
        bytes_in = bytearray(0)
        with open(filepath, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        obj = pickle.loads(bytes_in)
    except:
        return None
    return obj

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
PATH = "../csv/"
print(os.listdir(PATH))

['application_test.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
#read csv
POS_CASH_balance = import_data(PATH+'POS_CASH_balance.csv')
installments_payments = import_data(PATH+'installments_payments.csv')
credit_card_balance = import_data(PATH+'credit_card_balance.csv')
previous_application = import_data(PATH+'previous_application.csv')

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 171.69 MB
Decreased by 71.9%
Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%
Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 263.69 MB
Decreased by 60.9%
Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 130.62 MB
Decreased by 72.3%


In [4]:
#POS_CASH_balance
df1=POS_CASH_balance.sort_values("SK_ID_PREV")
df1 = df1.drop(columns="SK_ID_CURR")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_PREV","NAME_CONTRACT_STATUS"]).agg(['sum','max','mean'])
df2.head()

MONTHS_BALANCE           CNT_INSTALMENT        \
                                           sum max  mean            sum   max   
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000001    Active                        -19.0  -9  -9.5           24.0  12.0   
           Completed                      -8.0  -8  -8.0            2.0   2.0   
1000002    Active                       -210.0 -51 -52.5           22.0   6.0   
           Completed                     -50.0 -50 -50.0            4.0   4.0   
1000003    Active                        -10.0  -1  -2.5           48.0  12.0   

                                      CNT_INSTALMENT_FUTURE              \
                                 mean                   sum   max  mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                           
1000001    Active                12.0                  23.0  12.0  11.5   
           Completed              2.0                   0.0   0.0   0.0   
1000002    Active                 5.5                  10.0   4.0   2.5   
           Completed              4.0                   0.0   0.0   0.0   
1000003    Active                12.0                  42.0  12.0  10.5   

                                SK_DPD          SK_DPD_DEF          COUNT      \
                                   sum max mean        sum max mean   sum max   
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000001    Active                  0.0   0  0.0        0.0   0  0.0     2   1   
           Completed               0.0   0  0.0        0.0   0  0.0     1   1   
1000002    Active                  0.0   0  0.0        0.0   0  0.0     4   1   
           Completed               0.0   0  0.0        0.0   0  0.0     1   1   
1000003    Active                  0.0   0  0.0        0.0   0  0.0     4   1   

                                      
                                mean  
SK_ID_PREV NAME_CONTRACT_STATUS       
1000001    Active                  1  
           Completed               1  
1000002    Active                  1  
           Completed               1  
1000003    Active                  1

In [5]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("PC_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['PC_MONTHS_BALANCE_sum', 'PC_MONTHS_BALANCE_max', 'PC_MONTHS_BALANCE_mean', 'PC_CNT_INSTALMENT_sum', 'PC_CNT_INSTALMENT_max', 'PC_CNT_INSTALMENT_mean', 'PC_CNT_INSTALMENT_FUTURE_sum', 'PC_CNT_INSTALMENT_FUTURE_max', 'PC_CNT_INSTALMENT_FUTURE_mean', 'PC_SK_DPD_sum', 'PC_SK_DPD_max', 'PC_SK_DPD_mean', 'PC_SK_DPD_DEF_sum', 'PC_SK_DPD_DEF_max', 'PC_SK_DPD_DEF_mean', 'PC_COUNT_sum', 'PC_COUNT_max', 'PC_COUNT_mean']


PC_MONTHS_BALANCE_sum  PC_MONTHS_BALANCE_max  \
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000001    Active                                -19.0                     -9   
           Completed                              -8.0                     -8   
1000002    Active                               -210.0                    -51   
           Completed                             -50.0                    -50   
1000003    Active                                -10.0                     -1   

                                 PC_MONTHS_BALANCE_mean  \
SK_ID_PREV NAME_CONTRACT_STATUS                           
1000001    Active                                  -9.5   
           Completed                               -8.0   
1000002    Active                                 -52.5   
           Completed                              -50.0   
1000003    Active                                  -2.5   

                                 PC_CNT_INSTALMENT_sum  PC_CNT_INSTALMENT_max  \
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000001    Active                                 24.0                   12.0   
           Completed                               2.0                    2.0   
1000002    Active                                 22.0                    6.0   
           Completed                               4.0                    4.0   
1000003    Active                                 48.0                   12.0   

                                 PC_CNT_INSTALMENT_mean  \
SK_ID_PREV NAME_CONTRACT_STATUS                           
1000001    Active                                  12.0   
           Completed                                2.0   
1000002    Active                                   5.5   
           Completed                                4.0   
1000003    Active                                  12.0   

                                 PC_CNT_INSTALMENT_FUTURE_sum  \
SK_ID_PREV NAME_CONTRACT_STATUS                                 
1000001    Active                                        23.0   
           Completed                                      0.0   
1000002    Active                                        10.0   
           Completed                                      0.0   
1000003    Active                                        42.0   

                                 PC_CNT_INSTALMENT_FUTURE_max  \
SK_ID_PREV NAME_CONTRACT_STATUS                                 
1000001    Active                                        12.0   
           Completed                                      0.0   
1000002    Active                                         4.0   
           Completed                                      0.0   
1000003    Active                                        12.0   

                                 PC_CNT_INSTALMENT_FUTURE_mean  PC_SK_DPD_sum  \
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000001    Active                                         11.5            0.0   
           Completed                                       0.0            0.0   
1000002    Active                                          2.5            0.0   
           Completed                                       0.0            0.0   
1000003    Active                                         10.5            0.0   

                                 PC_SK_DPD_max  PC_SK_DPD_mean  \
SK_ID_PREV NAME_CONTRACT_STATUS                                  
1000001    Active                            0             0.0   
           Completed                         0             0.0   
1000002    Active                            0             0.0   
           Completed                         0             0.0   
1000003    Active                            0             0.0   

                                 PC_SK_DPD_DEF_sum  PC_SK_DPD_DEF_max  \
SK_ID_PREV NAME_CONTRACT_STATUS                      

In [6]:
df2_Active = df2.xs(key="Active",level=1)
df2_Active.columns = df2_Active.columns + "_Ac"

df2_Amortized_debt = df2.xs(key="Amortized debt",level=1)
df2_Amortized_debt.columns = df2_Amortized_debt.columns + "_Ad"

df2_Approved = df2.xs(key="Approved",level=1)
df2_Approved.columns = df2_Approved.columns + "_Ap"

df2_Canceled = df2.xs(key="Canceled",level=1)
df2_Canceled.columns = df2_Canceled.columns + "_Ca"

df2_Demand = df2.xs(key="Demand",level=1)
df2_Demand.columns = df2_Demand.columns + "_De"

df2_Returned_to_the_store = df2.xs(key="Returned to the store",level=1)
df2_Returned_to_the_store.columns = df2_Returned_to_the_store.columns + "_Re"

df2_Signed = df2.xs(key="Signed",level=1)
df2_Signed.columns = df2_Signed.columns + "_Si"

df2_XNA = df2.xs(key="XNA",level=1)
df2_XNA.columns = df2_XNA.columns + "_XN"

In [7]:
df3= pd.merge(df2_Active, df2_Amortized_debt, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Approved, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Canceled, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Demand, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Returned_to_the_store, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Signed, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_XNA, left_index=True, right_index=True, how='outer')
df_POS_CASH_balance = df3.reset_index()
df_POS_CASH_balance.head()

,SK_ID_PREV,PC_MONTHS_BALANCE_sum_Ac,PC_MONTHS_BALANCE_max_Ac,PC_MONTHS_BALANCE_mean_Ac,PC_CNT_INSTALMENT_sum_Ac,PC_CNT_INSTALMENT_max_Ac,PC_CNT_INSTALMENT_mean_Ac,PC_CNT_INSTALMENT_FUTURE_sum_Ac,PC_CNT_INSTALMENT_FUTURE_max_Ac,PC_CNT_INSTALMENT_FUTURE_mean_Ac,...,PC_CNT_INSTALMENT_FUTURE_mean_XN,PC_SK_DPD_sum_XN,PC_SK_DPD_max_XN,PC_SK_DPD_mean_XN,PC_SK_DPD_DEF_sum_XN,PC_SK_DPD_DEF_max_XN,PC_SK_DPD_DEF_mean_XN,PC_COUNT_sum_XN,PC_COUNT_max_XN,PC_COUNT_mean_XN
0,1000001,-19.0,-9.0,-9.5,24.0,12.0,12.0,23.0,12.0,11.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000002,-210.0,-51.0,-52.5,22.0,6.0,5.5,10.0,4.0,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000003,-10.0,-1.0,-2.5,48.0,12.0,12.0,42.0,12.0,10.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000004,-182.0,-23.0,-26.0,70.0,10.0,10.0,49.0,10.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000005,-515.0,-47.0,-51.5,100.0,10.0,10.0,55.0,10.0,5.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#installments_payments
df1=installments_payments.sort_values("SK_ID_PREV")
df1 = df1.drop(columns="SK_ID_CURR")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_PREV"]).agg(['sum','max','mean'])
df2.head(10)

NUM_INSTALMENT_VERSION                NUM_INSTALMENT_NUMBER      \
                              sum  max      mean                   sum max   
SK_ID_PREV                                                                   
1000001                       3.0  2.0  1.500000                   3.0   2   
1000002                       5.0  2.0  1.250000                  10.0   4   
1000003                       3.0  1.0  1.000000                   6.0   3   
1000004                       8.0  2.0  1.142578                  28.0   7   
1000005                      11.0  1.0  1.000000                  64.0  10   
1000007                       5.0  1.0  1.000000                  15.0   5   
1000008                      10.0  2.0  1.111328                  45.0   9   
1000009                       6.0  1.0  1.000000                  21.0   6   
1000010                      12.0  2.0  1.090820                  66.0  11   
1000011                      12.0  1.0  1.000000                  78.0  12   

                     DAYS_INSTALMENT                 DAYS_ENTRY_PAYMENT ...   \
                mean             sum     max    mean                sum ...    
SK_ID_PREV                                                              ...    
1000001     1.500000          -506.0  -238.0  -253.0             -538.0 ...    
1000002     2.500000         -6220.0 -1510.0 -1555.0            -6300.0 ...    
1000003     2.000000          -192.0   -34.0   -64.0             -238.0 ...    
1000004     4.000000         -5404.0  -682.0  -772.0            -5592.0 ...    
1000005     5.818182        -16976.0 -1418.0 -1543.0           -17072.0 ...    
1000007     3.000000          -315.0    -3.0   -63.0             -399.0 ...    
1000008     5.000000        -10456.0 -1042.0 -1162.0           -10784.0 ...    
1000009     3.500000         -2292.0  -307.0  -382.0            -2350.0 ...    
1000010     6.000000         -4488.0  -258.0  -408.0            -4600.0 ...    
1000011     6.500000         -3240.0  -105.0  -270.0            -3260.0 ...    

                      AMT_INSTALMENT                                \
                 mean            sum            max           mean   
SK_ID_PREV                                                           
1000001     -269.0000   6.844342e+04   62039.113281   34221.710938   
1000002    -1575.0000   3.723556e+04   18443.564453    9308.890625   
1000003      -79.3125   1.485405e+04    4951.350098    4951.350098   
1000004     -798.5000   3.352316e+04   13176.495117    4789.022461   
1000005    -1552.0000   1.617353e+05   14713.605469   14703.209961   
1000007      -79.8125   5.623402e+04   11246.804688   11246.804688   
1000008    -1199.0000   2.622386e+05   51585.300781   29137.621094   
1000009     -391.7500   5.581561e+04    9302.849609    9302.602539   
1000010     -418.2500   1.259663e+06  512843.125000  114514.828125   
1000011     -271.7500   1.109159e+06   92435.039062   92429.890625   

             AMT_PAYMENT                               COUNT           
                     sum            max           mean   sum max mean  
SK_ID_PREV                                                             
1000001     6.844342e+04   62039.113281   34221.710938     2   1    1  
1000002     3.723556e+04   18443.564453    9308.890625     4   1    1  
1000003     1.485405e+04    4951.350098    4951.350098     3   1    1  
1000004     3.352316e+04   13176.495117    4789.022461     7   1    1  
1000005     1.470217e+05   14713.605469   13365.610352    11   1    1  
1000007     5.623402e+04   11246.804688   11246.804688     5   1    1  
1000008     2.622386e+05   51585.300781   29137.621094     9   1    1  
1000009     5.581561e+04    9302.849609    9302.602539     6   1    1  
1000010     1.259663e+06  512843.125000  114514.828125    11   1    1  
1000011     1.109159e+06   92435.039062   92429.890625    12   1    1  

[10 rows x 21 columns]

In [9]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("insta_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['insta_NUM_INSTALMENT_VERSION_sum', 'insta_NUM_INSTALMENT_VERSION_max', 'insta_NUM_INSTALMENT_VERSION_mean', 'insta_NUM_INSTALMENT_NUMBER_sum', 'insta_NUM_INSTALMENT_NUMBER_max', 'insta_NUM_INSTALMENT_NUMBER_mean', 'insta_DAYS_INSTALMENT_sum', 'insta_DAYS_INSTALMENT_max', 'insta_DAYS_INSTALMENT_mean', 'insta_DAYS_ENTRY_PAYMENT_sum', 'insta_DAYS_ENTRY_PAYMENT_max', 'insta_DAYS_ENTRY_PAYMENT_mean', 'insta_AMT_INSTALMENT_sum', 'insta_AMT_INSTALMENT_max', 'insta_AMT_INSTALMENT_mean', 'insta_AMT_PAYMENT_sum', 'insta_AMT_PAYMENT_max', 'insta_AMT_PAYMENT_mean', 'insta_COUNT_sum', 'insta_COUNT_max', 'insta_COUNT_mean']


,insta_NUM_INSTALMENT_VERSION_sum,insta_NUM_INSTALMENT_VERSION_max,insta_NUM_INSTALMENT_VERSION_mean,insta_NUM_INSTALMENT_NUMBER_sum,insta_NUM_INSTALMENT_NUMBER_max,insta_NUM_INSTALMENT_NUMBER_mean,insta_DAYS_INSTALMENT_sum,insta_DAYS_INSTALMENT_max,insta_DAYS_INSTALMENT_mean,insta_DAYS_ENTRY_PAYMENT_sum,...,insta_DAYS_ENTRY_PAYMENT_mean,insta_AMT_INSTALMENT_sum,insta_AMT_INSTALMENT_max,insta_AMT_INSTALMENT_mean,insta_AMT_PAYMENT_sum,insta_AMT_PAYMENT_max,insta_AMT_PAYMENT_mean,insta_COUNT_sum,insta_COUNT_max,insta_COUNT_mean
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
1000001,3.0,2.0,1.500000,3.0,2,1.500000,-506.0,-238.0,-253.0,-538.0,...,-269.0000,68443.421875,62039.113281,34221.710938,68443.421875,62039.113281,34221.710938,2,1,1
1000002,5.0,2.0,1.250000,10.0,4,2.500000,-6220.0,-1510.0,-1555.0,-6300.0,...,-1575.0000,37235.562500,18443.564453,9308.890625,37235.562500,18443.564453,9308.890625,4,1,1
1000003,3.0,1.0,1.000000,6.0,3,2.000000,-192.0,-34.0,-64.0,-238.0,...,-79.3125,14854.050781,4951.350098,4951.350098,14854.050781,4951.350098,4951.350098,3,1,1
1000004,8.0,2.0,1.142578,28.0,7,4.000000,-5404.0,-682.0,-772.0,-5592.0,...,-798.5000,33523.156250,13176.495117,4789.022461,33523.156250,13176.495117,4789.022461,7,1,1
1000005,11.0,1.0,1.000000,64.0,10,5.818182,-16976.0,-1418.0,-1543.0,-17072.0,...,-1552.0000,161735.312500,14713.605469,14703.209961,147021.703125,14713.605469,13365.610352,11,1,1


In [10]:
df_installments_payments = df2.reset_index()
df_installments_payments.head()

,SK_ID_PREV,insta_NUM_INSTALMENT_VERSION_sum,insta_NUM_INSTALMENT_VERSION_max,insta_NUM_INSTALMENT_VERSION_mean,insta_NUM_INSTALMENT_NUMBER_sum,insta_NUM_INSTALMENT_NUMBER_max,insta_NUM_INSTALMENT_NUMBER_mean,insta_DAYS_INSTALMENT_sum,insta_DAYS_INSTALMENT_max,insta_DAYS_INSTALMENT_mean,...,insta_DAYS_ENTRY_PAYMENT_mean,insta_AMT_INSTALMENT_sum,insta_AMT_INSTALMENT_max,insta_AMT_INSTALMENT_mean,insta_AMT_PAYMENT_sum,insta_AMT_PAYMENT_max,insta_AMT_PAYMENT_mean,insta_COUNT_sum,insta_COUNT_max,insta_COUNT_mean
0,1000001,3.0,2.0,1.500000,3.0,2,1.500000,-506.0,-238.0,-253.0,...,-269.0000,68443.421875,62039.113281,34221.710938,68443.421875,62039.113281,34221.710938,2,1,1
1,1000002,5.0,2.0,1.250000,10.0,4,2.500000,-6220.0,-1510.0,-1555.0,...,-1575.0000,37235.562500,18443.564453,9308.890625,37235.562500,18443.564453,9308.890625,4,1,1
2,1000003,3.0,1.0,1.000000,6.0,3,2.000000,-192.0,-34.0,-64.0,...,-79.3125,14854.050781,4951.350098,4951.350098,14854.050781,4951.350098,4951.350098,3,1,1
3,1000004,8.0,2.0,1.142578,28.0,7,4.000000,-5404.0,-682.0,-772.0,...,-798.5000,33523.156250,13176.495117,4789.022461,33523.156250,13176.495117,4789.022461,7,1,1
4,1000005,11.0,1.0,1.000000,64.0,10,5.818182,-16976.0,-1418.0,-1543.0,...,-1552.0000,161735.312500,14713.605469,14703.209961,147021.703125,14713.605469,13365.610352,11,1,1


In [11]:
#credit_card_balance
df1=credit_card_balance.sort_values("SK_ID_PREV")
df1 = df1.drop(columns="SK_ID_CURR")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_PREV","NAME_CONTRACT_STATUS"]).agg(['sum','max','mean'])
df2.head()

MONTHS_BALANCE           AMT_BALANCE  \
                                           sum max mean          sum   
SK_ID_PREV NAME_CONTRACT_STATUS                                        
1000018    Active                        -20.0  -2 -4.0  374731.4375   
1000030    Active                        -36.0  -1 -4.5  447928.5000   
1000031    Active                       -136.0  -1 -8.5  838311.0000   
1000035    Active                        -20.0  -2 -4.0       0.0000   
1000077    Active                        -77.0  -2 -7.0       0.0000   

                                                             \
                                           max         mean   
SK_ID_PREV NAME_CONTRACT_STATUS                               
1000018    Active                136695.421875  74946.28125   
1000030    Active                103027.273438  55991.06250   
1000031    Active                154945.937500  52394.43750   
1000035    Active                     0.000000      0.00000   
1000077    Active                     0.000000      0.00000   

                                AMT_CREDIT_LIMIT_ACTUAL          \
                                                    sum     max   
SK_ID_PREV NAME_CONTRACT_STATUS                                   
1000018    Active                                405000  135000   
1000030    Active                                652500  135000   
1000031    Active                               2394000  225000   
1000035    Active                               1125000  225000   
1000077    Active                               1035000  135000   

                                               AMT_DRAWINGS_ATM_CURRENT ...   \
                                          mean                      sum ...    
SK_ID_PREV NAME_CONTRACT_STATUS                                         ...    
1000018    Active                 81000.000000                  27000.0 ...    
1000030    Active                 81562.500000                   4500.0 ...    
1000031    Active                149625.000000                 157500.0 ...    
1000035    Active                225000.000000                      0.0 ...    
1000077    Active                 94090.909091                      0.0 ...    

                                CNT_INSTALMENT_MATURE_CUM SK_DPD           \
                                                     mean    sum max mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                             
1000018    Active                                  2.0000    0.0   0  0.0   
1000030    Active                                  1.8750    0.0   0  0.0   
1000031    Active                                  3.6875    0.0   0  0.0   
1000035    Active                                  0.0000    0.0   0  0.0   
1000077    Active                                  0.0000    0.0   0  0.0   

                                SK_DPD_DEF          COUNT           
                                       sum max mean   sum max mean  
SK_ID_PREV NAME_CONTRACT_STATUS                                     
1000018    Active                      0.0   0  0.0     5   1    1  
1000030    Active                      0.0   0  0.0     8   1    1  
1000031    Active                      0.0   0  0.0    16   1    1  
1000035    Active                      0.0   0  0.0     5   1    1  
1000077    Active                      0.0   0  0.0    11   1    1  

[5 rows x 63 columns]

In [12]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("credit_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['credit_MONTHS_BALANCE_sum', 'credit_MONTHS_BALANCE_max', 'credit_MONTHS_BALANCE_mean', 'credit_AMT_BALANCE_sum', 'credit_AMT_BALANCE_max', 'credit_AMT_BALANCE_mean', 'credit_AMT_CREDIT_LIMIT_ACTUAL_sum', 'credit_AMT_CREDIT_LIMIT_ACTUAL_max', 'credit_AMT_CREDIT_LIMIT_ACTUAL_mean', 'credit_AMT_DRAWINGS_ATM_CURRENT_sum', 'credit_AMT_DRAWINGS_ATM_CURRENT_max', 'credit_AMT_DRAWINGS_ATM_CURRENT_mean', 'credit_AMT_DRAWINGS_CURRENT_sum', 'credit_AMT_DRAWINGS_CURRENT_max', 'credit_AMT_DRAWINGS_CURRENT_mean', 'credit_AMT_DRAWINGS_OTHER_CURRENT_sum', 'credit_AMT_DRAWINGS_OTHER_CURRENT_max', 'credit_AMT_DRAWINGS_OTHER_CURRENT_mean', 'credit_AMT_DRAWINGS_POS_CURRENT_sum', 'credit_AMT_DRAWINGS_POS_CURRENT_max', 'credit_AMT_DRAWINGS_POS_CURRENT_mean', 'credit_AMT_INST_MIN_REGULARITY_sum', 'credit_AMT_INST_MIN_REGULARITY_max', 'credit_AMT_INST_MIN_REGULARITY_mean', 'credit_AMT_PAYMENT_CURRENT_sum', 'credit_AMT_PAYMENT_CURRENT_max', 'credit_AMT_PAYMENT_CURRENT_mean', 'credit_AMT_PAYMENT_TOTAL_CURRENT

,,credit_MONTHS_BALANCE_sum,credit_MONTHS_BALANCE_max,credit_MONTHS_BALANCE_mean,credit_AMT_BALANCE_sum,credit_AMT_BALANCE_max,credit_AMT_BALANCE_mean,credit_AMT_CREDIT_LIMIT_ACTUAL_sum,credit_AMT_CREDIT_LIMIT_ACTUAL_max,credit_AMT_CREDIT_LIMIT_ACTUAL_mean,credit_AMT_DRAWINGS_ATM_CURRENT_sum,...,credit_CNT_INSTALMENT_MATURE_CUM_mean,credit_SK_DPD_sum,credit_SK_DPD_max,credit_SK_DPD_mean,credit_SK_DPD_DEF_sum,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_mean,credit_COUNT_sum,credit_COUNT_max,credit_COUNT_mean
SK_ID_PREV,NAME_CONTRACT_STATUS,,,,,,,,,,,,,,,,,,,,,
1000018,Active,-20.0,-2,-4.0,374731.4375,136695.421875,74946.28125,405000,135000,81000.000000,27000.0,...,2.0000,0.0,0,0.0,0.0,0,0.0,5,1,1
1000030,Active,-36.0,-1,-4.5,447928.5000,103027.273438,55991.06250,652500,135000,81562.500000,4500.0,...,1.8750,0.0,0,0.0,0.0,0,0.0,8,1,1
1000031,Active,-136.0,-1,-8.5,838311.0000,154945.937500,52394.43750,2394000,225000,149625.000000,157500.0,...,3.6875,0.0,0,0.0,0.0,0,0.0,16,1,1
1000035,Active,-20.0,-2,-4.0,0.0000,0.000000,0.00000,1125000,225000,225000.000000,0.0,...,0.0000,0.0,0,0.0,0.0,0,0.0,5,1,1
1000077,Active,-77.0,-2,-7.0,0.0000,0.000000,0.00000,1035000,135000,94090.909091,0.0,...,0.0000,0.0,0,0.0,0.0,0,0.0,11,1,1


In [13]:
df2_Active = df2.xs(key="Active",level=1)
df2_Active.columns = df2_Active.columns + "_Ac"

df2_Approved = df2.xs(key="Approved",level=1)
df2_Approved.columns = df2_Approved.columns + "_Ap"

df2_Completed = df2.xs(key="Completed",level=1)
df2_Completed.columns = df2_Completed.columns + "_Co"

df2_Demand = df2.xs(key="Demand",level=1)
df2_Demand.columns = df2_Demand.columns + "_De"

df2_Refused = df2.xs(key="Refused",level=1)
df2_Refused.columns = df2_Refused.columns + "_Re"

df2_Sent_proposal = df2.xs(key="Sent proposal",level=1)
df2_Sent_proposal.columns = df2_Sent_proposal.columns + "_Sp"

df2_Signed = df2.xs(key="Signed",level=1)
df2_Signed.columns = df2_Signed.columns + "_Si"

In [14]:
df3= pd.merge(df2_Active, df2_Approved, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Completed, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Demand, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Refused, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Sent_proposal, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Signed, left_index=True, right_index=True, how='outer')

In [15]:
df_credit_card_balance = df3.reset_index()
df_credit_card_balance.head()

,SK_ID_PREV,credit_MONTHS_BALANCE_sum_Ac,credit_MONTHS_BALANCE_max_Ac,credit_MONTHS_BALANCE_mean_Ac,credit_AMT_BALANCE_sum_Ac,credit_AMT_BALANCE_max_Ac,credit_AMT_BALANCE_mean_Ac,credit_AMT_CREDIT_LIMIT_ACTUAL_sum_Ac,credit_AMT_CREDIT_LIMIT_ACTUAL_max_Ac,credit_AMT_CREDIT_LIMIT_ACTUAL_mean_Ac,...,credit_CNT_INSTALMENT_MATURE_CUM_mean_Si,credit_SK_DPD_sum_Si,credit_SK_DPD_max_Si,credit_SK_DPD_mean_Si,credit_SK_DPD_DEF_sum_Si,credit_SK_DPD_DEF_max_Si,credit_SK_DPD_DEF_mean_Si,credit_COUNT_sum_Si,credit_COUNT_max_Si,credit_COUNT_mean_Si
0,1000018,-20.0,-2.0,-4.0,374731.4375,136695.421875,74946.28125,405000.0,135000.0,81000.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000030,-36.0,-1.0,-4.5,447928.5000,103027.273438,55991.06250,652500.0,135000.0,81562.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000031,-136.0,-1.0,-8.5,838311.0000,154945.937500,52394.43750,2394000.0,225000.0,149625.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000035,-20.0,-2.0,-4.0,0.0000,0.000000,0.00000,1125000.0,225000.0,225000.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000077,-77.0,-2.0,-7.0,0.0000,0.000000,0.00000,1035000.0,135000.0,94090.909091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#previous_application
previous_application.head()
previous_application['NAME_CONTRACT_TYPE'].value_counts()

Cash loans         747553
Consumer loans     729151
Revolving loans    193164
XNA                   346
Name: NAME_CONTRACT_TYPE, dtype: int64

In [17]:
pre_cashloan = previous_application[previous_application['NAME_CONTRACT_TYPE']=='Cash loans']
pre_consumer = previous_application[previous_application['NAME_CONTRACT_TYPE']=='Consumer loans']
pre_revolving = previous_application[previous_application['NAME_CONTRACT_TYPE']=='Revolving loans']
pre_XNA = previous_application[previous_application['NAME_CONTRACT_TYPE']=='XNA']

In [18]:
df1=pre_cashloan.sort_values("SK_ID_PREV")
df1 = df1.drop(columns="SK_ID_CURR")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_PREV","NAME_CONTRACT_STATUS"]).agg(['sum','max','mean'])
df2.head()

AMT_ANNUITY                              \
                                          sum           max          mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                             
1000010    Approved              74682.000000  74682.000000  74682.000000   
1000011    Approved              92435.039062  92435.039062  92435.039062   
1000027    Approved               8806.455078   8806.455078   8806.455078   
1000040    Approved               9686.339844   9686.339844   9686.339844   
1000050    Approved              13833.450195  13833.450195  13833.450195   

                                AMT_APPLICATION                      \
                                            sum       max      mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                       
1000010    Approved                    900000.0  900000.0  900000.0   
1000011    Approved                    855000.0  855000.0  855000.0   
1000027    Approved                     45000.0   45000.0   45000.0   
1000040    Approved                     45000.0   45000.0   45000.0   
1000050    Approved                    135000.0  135000.0  135000.0   

                                AMT_CREDIT                      \
                                       sum       max      mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                  
1000010    Approved               900000.0  900000.0  900000.0   
1000011    Approved               879831.0  879831.0  879831.0   
1000027    Approved                46485.0   46485.0   46485.0   
1000040    Approved                51898.5   51898.5   51898.5   
1000050    Approved               135000.0  135000.0  135000.0   

                                AMT_DOWN_PAYMENT ...  DAYS_LAST_DUE  \
                                             sum ...           mean   
SK_ID_PREV NAME_CONTRACT_STATUS                  ...                  
1000010    Approved                          0.0 ...         -258.0   
1000011    Approved                          0.0 ...         -105.0   
1000027    Approved                          0.0 ...         -225.0   
1000040    Approved                          0.0 ...         -746.0   
1000050    Approved                          0.0 ...       365243.0   

                                DAYS_TERMINATION                      \
                                             sum       max      mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                        
1000010    Approved                       -256.0    -256.0    -256.0   
1000011    Approved                       -101.0    -101.0    -101.0   
1000027    Approved                       -221.0    -221.0    -221.0   
1000040    Approved                       -743.0    -743.0    -743.0   
1000050    Approved                     365243.0  365243.0  365243.0   

                                NFLAG_INSURED_ON_APPROVAL           COUNT      \
                                                      sum  max mean   sum max   
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000010    Approved                                   0.0  0.0  0.0     1   1   
1000011    Approved                                   1.0  1.0  1.0     1   1   
1000027    Approved                                   1.0  1.0  1.0     1   1   
1000040    Approved                                   1.0  1.0  1.0     1   1   
1000050    Approved                                   0.0  0.0  0.0     1   1   

                                      
                                mean  
SK_ID_PREV NAME_CONTRACT_STATUS       
1000010    Approved                1  
1000011    Approved                1  
1000027    Approved                1  
1000040    Approved                1  
1000050    Approved                1  

[5 rows x 60 columns]

In [19]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("pre_cash_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['pre_cash_AMT_ANNUITY_sum', 'pre_cash_AMT_ANNUITY_max', 'pre_cash_AMT_ANNUITY_mean', 'pre_cash_AMT_APPLICATION_sum', 'pre_cash_AMT_APPLICATION_max', 'pre_cash_AMT_APPLICATION_mean', 'pre_cash_AMT_CREDIT_sum', 'pre_cash_AMT_CREDIT_max', 'pre_cash_AMT_CREDIT_mean', 'pre_cash_AMT_DOWN_PAYMENT_sum', 'pre_cash_AMT_DOWN_PAYMENT_max', 'pre_cash_AMT_DOWN_PAYMENT_mean', 'pre_cash_AMT_GOODS_PRICE_sum', 'pre_cash_AMT_GOODS_PRICE_max', 'pre_cash_AMT_GOODS_PRICE_mean', 'pre_cash_HOUR_APPR_PROCESS_START_sum', 'pre_cash_HOUR_APPR_PROCESS_START_max', 'pre_cash_HOUR_APPR_PROCESS_START_mean', 'pre_cash_NFLAG_LAST_APPL_IN_DAY_sum', 'pre_cash_NFLAG_LAST_APPL_IN_DAY_max', 'pre_cash_NFLAG_LAST_APPL_IN_DAY_mean', 'pre_cash_RATE_DOWN_PAYMENT_sum', 'pre_cash_RATE_DOWN_PAYMENT_max', 'pre_cash_RATE_DOWN_PAYMENT_mean', 'pre_cash_RATE_INTEREST_PRIMARY_sum', 'pre_cash_RATE_INTEREST_PRIMARY_max', 'pre_cash_RATE_INTEREST_PRIMARY_mean', 'pre_cash_RATE_INTEREST_PRIVILEGED_sum', 'pre_cash_RATE_INTEREST_PRIVILEGED_max',

,,pre_cash_AMT_ANNUITY_sum,pre_cash_AMT_ANNUITY_max,pre_cash_AMT_ANNUITY_mean,pre_cash_AMT_APPLICATION_sum,pre_cash_AMT_APPLICATION_max,pre_cash_AMT_APPLICATION_mean,pre_cash_AMT_CREDIT_sum,pre_cash_AMT_CREDIT_max,pre_cash_AMT_CREDIT_mean,pre_cash_AMT_DOWN_PAYMENT_sum,...,pre_cash_DAYS_LAST_DUE_mean,pre_cash_DAYS_TERMINATION_sum,pre_cash_DAYS_TERMINATION_max,pre_cash_DAYS_TERMINATION_mean,pre_cash_NFLAG_INSURED_ON_APPROVAL_sum,pre_cash_NFLAG_INSURED_ON_APPROVAL_max,pre_cash_NFLAG_INSURED_ON_APPROVAL_mean,pre_cash_COUNT_sum,pre_cash_COUNT_max,pre_cash_COUNT_mean
SK_ID_PREV,NAME_CONTRACT_STATUS,,,,,,,,,,,,,,,,,,,,,
1000010,Approved,74682.000000,74682.000000,74682.000000,900000.0,900000.0,900000.0,900000.0,900000.0,900000.0,0.0,...,-258.0,-256.0,-256.0,-256.0,0.0,0.0,0.0,1,1,1
1000011,Approved,92435.039062,92435.039062,92435.039062,855000.0,855000.0,855000.0,879831.0,879831.0,879831.0,0.0,...,-105.0,-101.0,-101.0,-101.0,1.0,1.0,1.0,1,1,1
1000027,Approved,8806.455078,8806.455078,8806.455078,45000.0,45000.0,45000.0,46485.0,46485.0,46485.0,0.0,...,-225.0,-221.0,-221.0,-221.0,1.0,1.0,1.0,1,1,1
1000040,Approved,9686.339844,9686.339844,9686.339844,45000.0,45000.0,45000.0,51898.5,51898.5,51898.5,0.0,...,-746.0,-743.0,-743.0,-743.0,1.0,1.0,1.0,1,1,1
1000050,Approved,13833.450195,13833.450195,13833.450195,135000.0,135000.0,135000.0,135000.0,135000.0,135000.0,0.0,...,365243.0,365243.0,365243.0,365243.0,0.0,0.0,0.0,1,1,1


In [20]:
pre_cashloan['NAME_CONTRACT_STATUS'].value_counts()

Approved        312540
Canceled        268591
Refused         165928
Unused offer       494
Name: NAME_CONTRACT_STATUS, dtype: int64

In [21]:
df2_Approved = df2.xs(key="Approved",level=1)
df2_Approved.columns = df2_Approved.columns + "_Ap"

df2_Canceled = df2.xs(key="Canceled",level=1)
df2_Canceled.columns = df2_Approved.columns + "_Ca"

df2_Refused = df2.xs(key="Refused",level=1)
df2_Refused.columns = df2_Refused.columns + "_Re"

df2_Unused = df2.xs(key="Unused offer",level=1)
df2_Unused.columns = df2_Unused.columns + "_Un"

In [22]:
df3= pd.merge(df2_Approved, df2_Canceled, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Refused, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Unused, left_index=True, right_index=True, how='outer')

In [23]:
df_pre_cash = df3.reset_index()
df_pre_cash.head()

,SK_ID_PREV,pre_cash_AMT_ANNUITY_sum_Ap,pre_cash_AMT_ANNUITY_max_Ap,pre_cash_AMT_ANNUITY_mean_Ap,pre_cash_AMT_APPLICATION_sum_Ap,pre_cash_AMT_APPLICATION_max_Ap,pre_cash_AMT_APPLICATION_mean_Ap,pre_cash_AMT_CREDIT_sum_Ap,pre_cash_AMT_CREDIT_max_Ap,pre_cash_AMT_CREDIT_mean_Ap,...,pre_cash_DAYS_LAST_DUE_mean_Un,pre_cash_DAYS_TERMINATION_sum_Un,pre_cash_DAYS_TERMINATION_max_Un,pre_cash_DAYS_TERMINATION_mean_Un,pre_cash_NFLAG_INSURED_ON_APPROVAL_sum_Un,pre_cash_NFLAG_INSURED_ON_APPROVAL_max_Un,pre_cash_NFLAG_INSURED_ON_APPROVAL_mean_Un,pre_cash_COUNT_sum_Un,pre_cash_COUNT_max_Un,pre_cash_COUNT_mean_Un
0,1000010,74682.000000,74682.000000,74682.000000,900000.0,900000.0,900000.0,900000.0,900000.0,900000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000011,92435.039062,92435.039062,92435.039062,855000.0,855000.0,855000.0,879831.0,879831.0,879831.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000027,8806.455078,8806.455078,8806.455078,45000.0,45000.0,45000.0,46485.0,46485.0,46485.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,9686.339844,9686.339844,9686.339844,45000.0,45000.0,45000.0,51898.5,51898.5,51898.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000050,13833.450195,13833.450195,13833.450195,135000.0,135000.0,135000.0,135000.0,135000.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df1=pre_consumer.sort_values("SK_ID_PREV")
df1 = df1.drop(columns="SK_ID_CURR")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_PREV","NAME_CONTRACT_STATUS"]).agg(['sum','max','mean'])
df2.head()

AMT_ANNUITY                              \
                                          sum           max          mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                             
1000001    Approved               6404.310059   6404.310059   6404.310059   
1000002    Approved               6264.000000   6264.000000   6264.000000   
1000003    Approved               4951.350098   4951.350098   4951.350098   
1000004    Approved               3391.110107   3391.110107   3391.110107   
1000005    Approved              14713.605469  14713.605469  14713.605469   

                                AMT_APPLICATION                                \
                                            sum            max           mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000001    Approved                58905.000000   58905.000000   58905.000000   
1000002    Approved                39145.500000   39145.500000   39145.500000   
1000003    Approved                47056.273438   47056.273438   47056.273438   
1000004    Approved                35144.371094   35144.371094   35144.371094   
1000005    Approved               123486.078125  123486.078125  123486.078125   

                                AMT_CREDIT                      \
                                       sum       max      mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                  
1000001    Approved                65124.0   65124.0   65124.0   
1000002    Approved                35230.5   35230.5   35230.5   
1000003    Approved                52641.0   52641.0   52641.0   
1000004    Approved                30586.5   30586.5   30586.5   
1000005    Approved               120307.5  120307.5  120307.5   

                                AMT_DOWN_PAYMENT ...  DAYS_LAST_DUE  \
                                             sum ...           mean   
SK_ID_PREV NAME_CONTRACT_STATUS                  ...                  
1000001    Approved                     0.000000 ...         -238.0   
1000002    Approved                  3915.000000 ...        -1510.0   
1000003    Approved                     4.275000 ...       365243.0   
1000004    Approved                  7032.870117 ...         -682.0   
1000005    Approved                 12349.575195 ...        -1418.0   

                                DAYS_TERMINATION                      \
                                             sum       max      mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                        
1000001    Approved                       -233.0    -233.0    -233.0   
1000002    Approved                      -1501.0   -1501.0   -1501.0   
1000003    Approved                     365243.0  365243.0  365243.0   
1000004    Approved                       -672.0    -672.0    -672.0   
1000005    Approved                      -1415.0   -1415.0   -1415.0   

                                NFLAG_INSURED_ON_APPROVAL           COUNT      \
                                                      sum  max mean   sum max   
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000001    Approved                                   0.0  0.0  0.0     1   1   
1000002    Approved                                   0.0  0.0  0.0     1   1   
1000003    Approved                                   1.0  1.0  1.0     1   1   
1000004    Approved                                   0.0  0.0  0.0     1   1   
1000005    Approved                                   0.0  0.0  0.0     1   1   

                                      
                                mean  
SK_ID_PREV NAME_CONTRACT_STATUS       
1000001    Approved                1  
1000002    Approved                1  
1000003    Approved                1  
1000004    Approved                1  
1000005    Approved                1  

[5 rows x 60 columns]

In [25]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("pre_consumer_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['pre_consumer_AMT_ANNUITY_sum', 'pre_consumer_AMT_ANNUITY_max', 'pre_consumer_AMT_ANNUITY_mean', 'pre_consumer_AMT_APPLICATION_sum', 'pre_consumer_AMT_APPLICATION_max', 'pre_consumer_AMT_APPLICATION_mean', 'pre_consumer_AMT_CREDIT_sum', 'pre_consumer_AMT_CREDIT_max', 'pre_consumer_AMT_CREDIT_mean', 'pre_consumer_AMT_DOWN_PAYMENT_sum', 'pre_consumer_AMT_DOWN_PAYMENT_max', 'pre_consumer_AMT_DOWN_PAYMENT_mean', 'pre_consumer_AMT_GOODS_PRICE_sum', 'pre_consumer_AMT_GOODS_PRICE_max', 'pre_consumer_AMT_GOODS_PRICE_mean', 'pre_consumer_HOUR_APPR_PROCESS_START_sum', 'pre_consumer_HOUR_APPR_PROCESS_START_max', 'pre_consumer_HOUR_APPR_PROCESS_START_mean', 'pre_consumer_NFLAG_LAST_APPL_IN_DAY_sum', 'pre_consumer_NFLAG_LAST_APPL_IN_DAY_max', 'pre_consumer_NFLAG_LAST_APPL_IN_DAY_mean', 'pre_consumer_RATE_DOWN_PAYMENT_sum', 'pre_consumer_RATE_DOWN_PAYMENT_max', 'pre_consumer_RATE_DOWN_PAYMENT_mean', 'pre_consumer_RATE_INTEREST_PRIMARY_sum', 'pre_consumer_RATE_INTEREST_PRIMARY_max', 'pre_consumer_RA

,,pre_consumer_AMT_ANNUITY_sum,pre_consumer_AMT_ANNUITY_max,pre_consumer_AMT_ANNUITY_mean,pre_consumer_AMT_APPLICATION_sum,pre_consumer_AMT_APPLICATION_max,pre_consumer_AMT_APPLICATION_mean,pre_consumer_AMT_CREDIT_sum,pre_consumer_AMT_CREDIT_max,pre_consumer_AMT_CREDIT_mean,pre_consumer_AMT_DOWN_PAYMENT_sum,...,pre_consumer_DAYS_LAST_DUE_mean,pre_consumer_DAYS_TERMINATION_sum,pre_consumer_DAYS_TERMINATION_max,pre_consumer_DAYS_TERMINATION_mean,pre_consumer_NFLAG_INSURED_ON_APPROVAL_sum,pre_consumer_NFLAG_INSURED_ON_APPROVAL_max,pre_consumer_NFLAG_INSURED_ON_APPROVAL_mean,pre_consumer_COUNT_sum,pre_consumer_COUNT_max,pre_consumer_COUNT_mean
SK_ID_PREV,NAME_CONTRACT_STATUS,,,,,,,,,,,,,,,,,,,,,
1000001,Approved,6404.310059,6404.310059,6404.310059,58905.000000,58905.000000,58905.000000,65124.0,65124.0,65124.0,0.000000,...,-238.0,-233.0,-233.0,-233.0,0.0,0.0,0.0,1,1,1
1000002,Approved,6264.000000,6264.000000,6264.000000,39145.500000,39145.500000,39145.500000,35230.5,35230.5,35230.5,3915.000000,...,-1510.0,-1501.0,-1501.0,-1501.0,0.0,0.0,0.0,1,1,1
1000003,Approved,4951.350098,4951.350098,4951.350098,47056.273438,47056.273438,47056.273438,52641.0,52641.0,52641.0,4.275000,...,365243.0,365243.0,365243.0,365243.0,1.0,1.0,1.0,1,1,1
1000004,Approved,3391.110107,3391.110107,3391.110107,35144.371094,35144.371094,35144.371094,30586.5,30586.5,30586.5,7032.870117,...,-682.0,-672.0,-672.0,-672.0,0.0,0.0,0.0,1,1,1
1000005,Approved,14713.605469,14713.605469,14713.605469,123486.078125,123486.078125,123486.078125,120307.5,120307.5,120307.5,12349.575195,...,-1418.0,-1415.0,-1415.0,-1415.0,0.0,0.0,0.0,1,1,1


In [26]:
pre_consumer['NAME_CONTRACT_STATUS'].value_counts()

Approved        626470
Refused          75185
Unused offer     25937
Canceled          1559
Name: NAME_CONTRACT_STATUS, dtype: int64

In [27]:
df2_Approved = df2.xs(key="Approved",level=1)
df2_Approved.columns = df2_Approved.columns + "_Ap"

df2_Canceled = df2.xs(key="Canceled",level=1)
df2_Canceled.columns = df2_Approved.columns + "_Ca"

df2_Refused = df2.xs(key="Refused",level=1)
df2_Refused.columns = df2_Refused.columns + "_Re"

df2_Unused = df2.xs(key="Unused offer",level=1)
df2_Unused.columns = df2_Unused.columns + "_Un"

In [28]:
df3= pd.merge(df2_Approved, df2_Canceled, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Refused, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Unused, left_index=True, right_index=True, how='outer')

In [29]:
df_pre_consumer = df3.reset_index()
df_pre_consumer.head()

,SK_ID_PREV,pre_consumer_AMT_ANNUITY_sum_Ap,pre_consumer_AMT_ANNUITY_max_Ap,pre_consumer_AMT_ANNUITY_mean_Ap,pre_consumer_AMT_APPLICATION_sum_Ap,pre_consumer_AMT_APPLICATION_max_Ap,pre_consumer_AMT_APPLICATION_mean_Ap,pre_consumer_AMT_CREDIT_sum_Ap,pre_consumer_AMT_CREDIT_max_Ap,pre_consumer_AMT_CREDIT_mean_Ap,...,pre_consumer_DAYS_LAST_DUE_mean_Un,pre_consumer_DAYS_TERMINATION_sum_Un,pre_consumer_DAYS_TERMINATION_max_Un,pre_consumer_DAYS_TERMINATION_mean_Un,pre_consumer_NFLAG_INSURED_ON_APPROVAL_sum_Un,pre_consumer_NFLAG_INSURED_ON_APPROVAL_max_Un,pre_consumer_NFLAG_INSURED_ON_APPROVAL_mean_Un,pre_consumer_COUNT_sum_Un,pre_consumer_COUNT_max_Un,pre_consumer_COUNT_mean_Un
0,1000001,6404.310059,6404.310059,6404.310059,58905.000000,58905.000000,58905.000000,65124.0,65124.0,65124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000002,6264.000000,6264.000000,6264.000000,39145.500000,39145.500000,39145.500000,35230.5,35230.5,35230.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000003,4951.350098,4951.350098,4951.350098,47056.273438,47056.273438,47056.273438,52641.0,52641.0,52641.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000004,3391.110107,3391.110107,3391.110107,35144.371094,35144.371094,35144.371094,30586.5,30586.5,30586.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000005,14713.605469,14713.605469,14713.605469,123486.078125,123486.078125,123486.078125,120307.5,120307.5,120307.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df1=pre_revolving.sort_values("SK_ID_PREV")
df1 = df1.drop(columns="SK_ID_CURR")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_PREV","NAME_CONTRACT_STATUS"]).agg(['sum','max','mean'])
df2.head()

AMT_ANNUITY                 AMT_APPLICATION  \
                                        sum     max    mean             sum   
SK_ID_PREV NAME_CONTRACT_STATUS                                               
1000018    Approved                  2250.0  2250.0  2250.0         45000.0   
1000030    Approved                  2250.0  2250.0  2250.0         45000.0   
1000031    Approved                  2250.0  2250.0  2250.0         45000.0   
1000035    Approved                  2250.0  2250.0  2250.0         45000.0   
1000077    Approved                  2250.0  2250.0  2250.0         45000.0   

                                                  AMT_CREDIT           \
                                     max     mean        sum      max   
SK_ID_PREV NAME_CONTRACT_STATUS                                         
1000018    Approved              45000.0  45000.0    45000.0  45000.0   
1000030    Approved              45000.0  45000.0    45000.0  45000.0   
1000031    Approved              45000.0  45000.0    45000.0  45000.0   
1000035    Approved              45000.0  45000.0    45000.0  45000.0   
1000077    Approved              45000.0  45000.0    45000.0  45000.0   

                                         AMT_DOWN_PAYMENT ...  DAYS_LAST_DUE  \
                                    mean              sum ...           mean   
SK_ID_PREV NAME_CONTRACT_STATUS                           ...                  
1000018    Approved              45000.0              0.0 ...       365243.0   
1000030    Approved              45000.0              0.0 ...       365243.0   
1000031    Approved              45000.0              0.0 ...       365243.0   
1000035    Approved              45000.0              0.0 ...       365243.0   
1000077    Approved              45000.0              0.0 ...       365243.0   

                                DAYS_TERMINATION                      \
                                             sum       max      mean   
SK_ID_PREV NAME_CONTRACT_STATUS                                        
1000018    Approved                     365243.0  365243.0  365243.0   
1000030    Approved                     365243.0  365243.0  365243.0   
1000031    Approved                     365243.0  365243.0  365243.0   
1000035    Approved                     365243.0  365243.0  365243.0   
1000077    Approved                     365243.0  365243.0  365243.0   

                                NFLAG_INSURED_ON_APPROVAL           COUNT      \
                                                      sum  max mean   sum max   
SK_ID_PREV NAME_CONTRACT_STATUS                                                 
1000018    Approved                                   0.0  0.0  0.0     1   1   
1000030    Approved                                   0.0  0.0  0.0     1   1   
1000031    Approved                                   0.0  0.0  0.0     1   1   
1000035    Approved                                   0.0  0.0  0.0     1   1   
1000077    Approved                                   0.0  0.0  0.0     1   1   

                                      
                                mean  
SK_ID_PREV NAME_CONTRACT_STATUS       
1000018    Approved                1  
1000030    Approved                1  
1000031    Approved                1  
1000035    Approved                1  
1000077    Approved                1  

[5 rows x 60 columns]

In [31]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("pre_revo_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['pre_revo_AMT_ANNUITY_sum', 'pre_revo_AMT_ANNUITY_max', 'pre_revo_AMT_ANNUITY_mean', 'pre_revo_AMT_APPLICATION_sum', 'pre_revo_AMT_APPLICATION_max', 'pre_revo_AMT_APPLICATION_mean', 'pre_revo_AMT_CREDIT_sum', 'pre_revo_AMT_CREDIT_max', 'pre_revo_AMT_CREDIT_mean', 'pre_revo_AMT_DOWN_PAYMENT_sum', 'pre_revo_AMT_DOWN_PAYMENT_max', 'pre_revo_AMT_DOWN_PAYMENT_mean', 'pre_revo_AMT_GOODS_PRICE_sum', 'pre_revo_AMT_GOODS_PRICE_max', 'pre_revo_AMT_GOODS_PRICE_mean', 'pre_revo_HOUR_APPR_PROCESS_START_sum', 'pre_revo_HOUR_APPR_PROCESS_START_max', 'pre_revo_HOUR_APPR_PROCESS_START_mean', 'pre_revo_NFLAG_LAST_APPL_IN_DAY_sum', 'pre_revo_NFLAG_LAST_APPL_IN_DAY_max', 'pre_revo_NFLAG_LAST_APPL_IN_DAY_mean', 'pre_revo_RATE_DOWN_PAYMENT_sum', 'pre_revo_RATE_DOWN_PAYMENT_max', 'pre_revo_RATE_DOWN_PAYMENT_mean', 'pre_revo_RATE_INTEREST_PRIMARY_sum', 'pre_revo_RATE_INTEREST_PRIMARY_max', 'pre_revo_RATE_INTEREST_PRIMARY_mean', 'pre_revo_RATE_INTEREST_PRIVILEGED_sum', 'pre_revo_RATE_INTEREST_PRIVILEGED_max',

,,pre_revo_AMT_ANNUITY_sum,pre_revo_AMT_ANNUITY_max,pre_revo_AMT_ANNUITY_mean,pre_revo_AMT_APPLICATION_sum,pre_revo_AMT_APPLICATION_max,pre_revo_AMT_APPLICATION_mean,pre_revo_AMT_CREDIT_sum,pre_revo_AMT_CREDIT_max,pre_revo_AMT_CREDIT_mean,pre_revo_AMT_DOWN_PAYMENT_sum,...,pre_revo_DAYS_LAST_DUE_mean,pre_revo_DAYS_TERMINATION_sum,pre_revo_DAYS_TERMINATION_max,pre_revo_DAYS_TERMINATION_mean,pre_revo_NFLAG_INSURED_ON_APPROVAL_sum,pre_revo_NFLAG_INSURED_ON_APPROVAL_max,pre_revo_NFLAG_INSURED_ON_APPROVAL_mean,pre_revo_COUNT_sum,pre_revo_COUNT_max,pre_revo_COUNT_mean
SK_ID_PREV,NAME_CONTRACT_STATUS,,,,,,,,,,,,,,,,,,,,,
1000018,Approved,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,0.0,...,365243.0,365243.0,365243.0,365243.0,0.0,0.0,0.0,1,1,1
1000030,Approved,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,0.0,...,365243.0,365243.0,365243.0,365243.0,0.0,0.0,0.0,1,1,1
1000031,Approved,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,0.0,...,365243.0,365243.0,365243.0,365243.0,0.0,0.0,0.0,1,1,1
1000035,Approved,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,0.0,...,365243.0,365243.0,365243.0,365243.0,0.0,0.0,0.0,1,1,1
1000077,Approved,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,0.0,...,365243.0,365243.0,365243.0,365243.0,0.0,0.0,0.0,1,1,1


In [32]:
pre_revolving['NAME_CONTRACT_STATUS'].value_counts()

Approved        97771
Refused         49534
Canceled        45854
Unused offer        5
Name: NAME_CONTRACT_STATUS, dtype: int64

In [33]:
df2_Approved = df2.xs(key="Approved",level=1)
df2_Approved.columns = df2_Approved.columns + "_Ap"

df2_Canceled = df2.xs(key="Canceled",level=1)
df2_Canceled.columns = df2_Approved.columns + "_Ca"

df2_Refused = df2.xs(key="Refused",level=1)
df2_Refused.columns = df2_Refused.columns + "_Re"

df2_Unused = df2.xs(key="Unused offer",level=1)
df2_Unused.columns = df2_Unused.columns + "_Un"

In [34]:
df3= pd.merge(df2_Approved, df2_Canceled, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Refused, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Unused, left_index=True, right_index=True, how='outer')

In [35]:
df_pre_revolving = df3.reset_index()
df_pre_revolving.head()

,SK_ID_PREV,pre_revo_AMT_ANNUITY_sum_Ap,pre_revo_AMT_ANNUITY_max_Ap,pre_revo_AMT_ANNUITY_mean_Ap,pre_revo_AMT_APPLICATION_sum_Ap,pre_revo_AMT_APPLICATION_max_Ap,pre_revo_AMT_APPLICATION_mean_Ap,pre_revo_AMT_CREDIT_sum_Ap,pre_revo_AMT_CREDIT_max_Ap,pre_revo_AMT_CREDIT_mean_Ap,...,pre_revo_DAYS_LAST_DUE_mean_Un,pre_revo_DAYS_TERMINATION_sum_Un,pre_revo_DAYS_TERMINATION_max_Un,pre_revo_DAYS_TERMINATION_mean_Un,pre_revo_NFLAG_INSURED_ON_APPROVAL_sum_Un,pre_revo_NFLAG_INSURED_ON_APPROVAL_max_Un,pre_revo_NFLAG_INSURED_ON_APPROVAL_mean_Un,pre_revo_COUNT_sum_Un,pre_revo_COUNT_max_Un,pre_revo_COUNT_mean_Un
0,1000018,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000030,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000031,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000035,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000077,2250.0,2250.0,2250.0,45000.0,45000.0,45000.0,45000.0,45000.0,45000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df1=pre_XNA.sort_values("SK_ID_PREV")
df1 = df1.drop(columns="SK_ID_CURR")
df1["COUNT"] = 1
df2 = df1.groupby(["SK_ID_PREV","NAME_CONTRACT_STATUS"]).agg(['sum','max','mean'])
df2.head()

AMT_ANNUITY          AMT_APPLICATION       \
                                        sum max mean             sum  max   
SK_ID_PREV NAME_CONTRACT_STATUS                                             
1016448    Canceled                     0.0 NaN  NaN             0.0  0.0   
1018815    Refused                      0.0 NaN  NaN             0.0  0.0   
1022801    Canceled                     0.0 NaN  NaN             0.0  0.0   
1037118    Canceled                     0.0 NaN  NaN             0.0  0.0   
1038412    Canceled                     0.0 NaN  NaN             0.0  0.0   

                                     AMT_CREDIT           AMT_DOWN_PAYMENT  \
                                mean        sum  max mean              sum   
SK_ID_PREV NAME_CONTRACT_STATUS                                              
1016448    Canceled              0.0        0.0  0.0  0.0              0.0   
1018815    Refused               0.0        0.0  0.0  0.0              0.0   
1022801    Canceled              0.0        0.0  0.0  0.0              0.0   
1037118    Canceled              0.0        0.0  0.0  0.0              0.0   
1038412    Canceled              0.0        0.0  0.0  0.0              0.0   

                                ...  DAYS_LAST_DUE DAYS_TERMINATION           \
                                ...           mean              sum max mean   
SK_ID_PREV NAME_CONTRACT_STATUS ...                                            
1016448    Canceled             ...            NaN              0.0 NaN  NaN   
1018815    Refused              ...            NaN              0.0 NaN  NaN   
1022801    Canceled             ...            NaN              0.0 NaN  NaN   
1037118    Canceled             ...            NaN              0.0 NaN  NaN   
1038412    Canceled             ...            NaN              0.0 NaN  NaN   

                                NFLAG_INSURED_ON_APPROVAL          COUNT      \
                                                      sum max mean   sum max   
SK_ID_PREV NAME_CONTRACT_STATUS                                                
1016448    Canceled                                   0.0 NaN  NaN     1   1   
1018815    Refused                                    0.0 NaN  NaN     1   1   
1022801    Canceled                                   0.0 NaN  NaN     1   1   
1037118    Canceled                                   0.0 NaN  NaN     1   1   
1038412    Canceled                                   0.0 NaN  NaN     1   1   

                                      
                                mean  
SK_ID_PREV NAME_CONTRACT_STATUS       
1016448    Canceled                1  
1018815    Refused                 1  
1022801    Canceled                1  
1037118    Canceled                1  
1038412    Canceled                1  

[5 rows x 60 columns]

In [37]:
columns_df2 = list(df2.columns)
varname = []
for i in range(0,len(columns_df2)):
    varname.append("pre_revo_" + "_".join(columns_df2[i]))
print(varname)
df2.columns =varname
df2.head()

['pre_revo_AMT_ANNUITY_sum', 'pre_revo_AMT_ANNUITY_max', 'pre_revo_AMT_ANNUITY_mean', 'pre_revo_AMT_APPLICATION_sum', 'pre_revo_AMT_APPLICATION_max', 'pre_revo_AMT_APPLICATION_mean', 'pre_revo_AMT_CREDIT_sum', 'pre_revo_AMT_CREDIT_max', 'pre_revo_AMT_CREDIT_mean', 'pre_revo_AMT_DOWN_PAYMENT_sum', 'pre_revo_AMT_DOWN_PAYMENT_max', 'pre_revo_AMT_DOWN_PAYMENT_mean', 'pre_revo_AMT_GOODS_PRICE_sum', 'pre_revo_AMT_GOODS_PRICE_max', 'pre_revo_AMT_GOODS_PRICE_mean', 'pre_revo_HOUR_APPR_PROCESS_START_sum', 'pre_revo_HOUR_APPR_PROCESS_START_max', 'pre_revo_HOUR_APPR_PROCESS_START_mean', 'pre_revo_NFLAG_LAST_APPL_IN_DAY_sum', 'pre_revo_NFLAG_LAST_APPL_IN_DAY_max', 'pre_revo_NFLAG_LAST_APPL_IN_DAY_mean', 'pre_revo_RATE_DOWN_PAYMENT_sum', 'pre_revo_RATE_DOWN_PAYMENT_max', 'pre_revo_RATE_DOWN_PAYMENT_mean', 'pre_revo_RATE_INTEREST_PRIMARY_sum', 'pre_revo_RATE_INTEREST_PRIMARY_max', 'pre_revo_RATE_INTEREST_PRIMARY_mean', 'pre_revo_RATE_INTEREST_PRIVILEGED_sum', 'pre_revo_RATE_INTEREST_PRIVILEGED_max',

,,pre_revo_AMT_ANNUITY_sum,pre_revo_AMT_ANNUITY_max,pre_revo_AMT_ANNUITY_mean,pre_revo_AMT_APPLICATION_sum,pre_revo_AMT_APPLICATION_max,pre_revo_AMT_APPLICATION_mean,pre_revo_AMT_CREDIT_sum,pre_revo_AMT_CREDIT_max,pre_revo_AMT_CREDIT_mean,pre_revo_AMT_DOWN_PAYMENT_sum,...,pre_revo_DAYS_LAST_DUE_mean,pre_revo_DAYS_TERMINATION_sum,pre_revo_DAYS_TERMINATION_max,pre_revo_DAYS_TERMINATION_mean,pre_revo_NFLAG_INSURED_ON_APPROVAL_sum,pre_revo_NFLAG_INSURED_ON_APPROVAL_max,pre_revo_NFLAG_INSURED_ON_APPROVAL_mean,pre_revo_COUNT_sum,pre_revo_COUNT_max,pre_revo_COUNT_mean
SK_ID_PREV,NAME_CONTRACT_STATUS,,,,,,,,,,,,,,,,,,,,,
1016448,Canceled,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1,1,1
1018815,Refused,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1,1,1
1022801,Canceled,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1,1,1
1037118,Canceled,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1,1,1
1038412,Canceled,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1,1,1


In [38]:
pre_XNA['NAME_CONTRACT_STATUS'].value_counts()

Canceled        315
Refused          31
Unused offer      0
Approved          0
Name: NAME_CONTRACT_STATUS, dtype: int64

In [39]:
df2_Approved = df2.xs(key="Approved",level=1)
df2_Approved.columns = df2_Approved.columns + "_Ap"

df2_Canceled = df2.xs(key="Canceled",level=1)
df2_Canceled.columns = df2_Canceled.columns + "_Ca"

df2_Refused = df2.xs(key="Refused",level=1)
df2_Refused.columns = df2_Refused.columns + "_Re"

df2_Unused = df2.xs(key="Unused offer",level=1)
df2_Unused.columns = df2_Unused.columns + "_Un"

In [40]:
df3= pd.merge(df2_Approved, df2_Canceled, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Refused, left_index=True, right_index=True, how='outer')
df3= pd.merge(df3, df2_Unused, left_index=True, right_index=True, how='outer')

In [41]:
df_pre_xna = df3.reset_index()
df_pre_xna.head()

,SK_ID_PREV,pre_revo_AMT_ANNUITY_sum_Ap,pre_revo_AMT_ANNUITY_max_Ap,pre_revo_AMT_ANNUITY_mean_Ap,pre_revo_AMT_APPLICATION_sum_Ap,pre_revo_AMT_APPLICATION_max_Ap,pre_revo_AMT_APPLICATION_mean_Ap,pre_revo_AMT_CREDIT_sum_Ap,pre_revo_AMT_CREDIT_max_Ap,pre_revo_AMT_CREDIT_mean_Ap,...,pre_revo_DAYS_LAST_DUE_mean_Un,pre_revo_DAYS_TERMINATION_sum_Un,pre_revo_DAYS_TERMINATION_max_Un,pre_revo_DAYS_TERMINATION_mean_Un,pre_revo_NFLAG_INSURED_ON_APPROVAL_sum_Un,pre_revo_NFLAG_INSURED_ON_APPROVAL_max_Un,pre_revo_NFLAG_INSURED_ON_APPROVAL_mean_Un,pre_revo_COUNT_sum_Un,pre_revo_COUNT_max_Un,pre_revo_COUNT_mean_Un
0,1016448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1018815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1022801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1037118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1038412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df4= pd.merge(df_pre_cash, df_pre_consumer, on="SK_ID_PREV", how='outer')
df4= pd.merge(df4, df_pre_revolving, on="SK_ID_PREV", how='outer')
df4= pd.merge(df4, df_pre_xna, on="SK_ID_PREV", how='outer')

In [43]:
df_pre = df4
df_pre.head()

,SK_ID_PREV,pre_cash_AMT_ANNUITY_sum_Ap,pre_cash_AMT_ANNUITY_max_Ap,pre_cash_AMT_ANNUITY_mean_Ap,pre_cash_AMT_APPLICATION_sum_Ap,pre_cash_AMT_APPLICATION_max_Ap,pre_cash_AMT_APPLICATION_mean_Ap,pre_cash_AMT_CREDIT_sum_Ap,pre_cash_AMT_CREDIT_max_Ap,pre_cash_AMT_CREDIT_mean_Ap,...,pre_revo_DAYS_LAST_DUE_mean_Un_y,pre_revo_DAYS_TERMINATION_sum_Un_y,pre_revo_DAYS_TERMINATION_max_Un_y,pre_revo_DAYS_TERMINATION_mean_Un_y,pre_revo_NFLAG_INSURED_ON_APPROVAL_sum_Un_y,pre_revo_NFLAG_INSURED_ON_APPROVAL_max_Un_y,pre_revo_NFLAG_INSURED_ON_APPROVAL_mean_Un_y,pre_revo_COUNT_sum_Un_y,pre_revo_COUNT_max_Un_y,pre_revo_COUNT_mean_Un_y
0,1000010,74682.000000,74682.000000,74682.000000,900000.0,900000.0,900000.0,900000.0,900000.0,900000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000011,92435.039062,92435.039062,92435.039062,855000.0,855000.0,855000.0,879831.0,879831.0,879831.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000027,8806.455078,8806.455078,8806.455078,45000.0,45000.0,45000.0,46485.0,46485.0,46485.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,9686.339844,9686.339844,9686.339844,45000.0,45000.0,45000.0,51898.5,51898.5,51898.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000050,13833.450195,13833.450195,13833.450195,135000.0,135000.0,135000.0,135000.0,135000.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#SK_ID_PREV & SK_ID_CURR
key_pre = previous_application[["SK_ID_CURR","SK_ID_PREV"]]
print(key_pre.duplicated(subset='SK_ID_PREV').value_counts())
print(key_pre.duplicated(subset='SK_ID_CURR').value_counts())

False    1670214
dtype: int64
True     1331357
False     338857
dtype: int64


In [45]:
df5= pd.merge(key_pre, df_pre, on="SK_ID_PREV", how='outer')
df5= pd.merge(df5, df_credit_card_balance, on="SK_ID_PREV", how='outer')
df5= pd.merge(df5, df_installments_payments, on="SK_ID_PREV", how='outer')
df5= pd.merge(df5, df_POS_CASH_balance, on="SK_ID_PREV", how='outer')

In [46]:
df6 = df5.groupby(by="SK_ID_CURR").sum()
df6 = df6.drop(columns="SK_ID_PREV")

previous_data = df6.reset_index()
previous_data.head()

,SK_ID_CURR,pre_cash_AMT_ANNUITY_sum_Ap,pre_cash_AMT_ANNUITY_max_Ap,pre_cash_AMT_ANNUITY_mean_Ap,pre_cash_AMT_APPLICATION_sum_Ap,pre_cash_AMT_APPLICATION_max_Ap,pre_cash_AMT_APPLICATION_mean_Ap,pre_cash_AMT_CREDIT_sum_Ap,pre_cash_AMT_CREDIT_max_Ap,pre_cash_AMT_CREDIT_mean_Ap,...,PC_CNT_INSTALMENT_FUTURE_mean_XN,PC_SK_DPD_sum_XN,PC_SK_DPD_max_XN,PC_SK_DPD_mean_XN,PC_SK_DPD_DEF_sum_XN,PC_SK_DPD_DEF_max_XN,PC_SK_DPD_DEF_mean_XN,PC_COUNT_sum_XN,PC_COUNT_max_XN,PC_COUNT_mean_XN
0,100001.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100002.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100003.0,98356.992188,98356.992188,98356.992188,900000.0,900000.0,900000.0,1035882.0,1035882.0,1035882.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100004.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100005.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
#save pkl
save_as_pickled_object(previous_data, '/Volumes/sub/kaggle/pkl/Home Credit Default Risk/previous_data.pkl')